In [160]:
import math
import numpy as np
import matplotlib.pyplot as plt 
from graphviz import Digraph
from IPython.display import display



In [161]:
class Value:
    def __init__(self, data, _children=(), _op='', label='', grad=0.0):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.label = label
        self.grad = grad
        self._backward = lambda: None  
    def __repr__(self):
        return f"Value(data={self.data})"
    def __add__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out
    def __mul__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    def __pow__(self, other):
        out = Value(self.data ** other, (self,), '^')
        def _backward():
            self.grad += (other * self.data ** (other - 1)) * out.grad
        out._backward = _backward
        return out
    def exp(self):
        out=Value(math.exp(self.data),(self,),'exp')
        def _backward():
            self.grad += out.data * out.grad
        out._backward=_backward
        return out
    def __sub__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        out = Value(self.data - other.data, (self, other), '-')
        def _backward():
            self.grad += out.grad
            other.grad += -1 * out.grad
        out._backward = _backward
        return out
    def __rmul__(self,other):
        return self*other
    def __truediv__(self,other):
        return self*(other**-1)
    def __neg__(self):
        return self * -1
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [162]:
def build(root):
    nodes,edges=set(),set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child,v))
                build(child)
    build(root)
    return nodes,edges
def draw_dot(root):
    dot=Digraph(format='svg',graph_attr={'rankdir':'LR'})
    nodes,edges=build(root)
    for n in nodes:
        uid=str(id(n))
        dot.node(name=uid,label="{ %s | data %.4f | grad %.4f}"%(n.label,n.data,n.grad),shape='record')
        if n._op:
            dot.node(name=uid + n._op,label=n._op)
            dot.edge(uid + n._op,uid)
    for n1,n2 in edges:
        dot.edge(str(id(n1)),str(id(n2)) + n2._op)
    return dot

In [163]:
class Neuron:
    def __init__(self,nin):
        self.w=[(Value(np.random.uniform(-1,1))) for _ in range(nin)]
        self.b=Value(np.random.uniform(-1,1))
    def __call__(self, x):
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        e_pos = act.exp()
        e_neg = (-act).exp()
        intermidate = (e_pos - e_neg) / (e_pos + e_neg)
        return intermidate
    def parameters(self):
        return self.w + [self.b]

class Layer:
    def __init__(self,nin,nout):
        self.neurons=[Neuron(nin) for _ in range(nout)]
    def __call__(self,x):
        out=[n(x) for n in self.neurons]
        return out[0] if len(out)==1 else out
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
class MLP:
    def __init__(self,nlayer,nin):
        sz=[nlayer]
        sz.extend(nin)
        self.layers=[Layer(sz[i],sz[i+1]) for i in range(len(sz)-1)]
    def __call__(self,x):
        for layer in self.layers:
            x=layer(x)
        return x
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]
        

In [759]:
n=MLP(3,[4,4,1])
parms=n.parameters()
x=np.array([[2.0,-1.0,3.0],[1.0,3.0,-1.0],[2.0,0.0,1.0],[5.4,1.2,2.7]])
y=np.array([1.0,-1.0,1.0,-1.0])


In [763]:
for i in range(1000):
    y_pred=[n(i) for i in x]
    loss= sum(((y_pred - y)**2 for y_pred,y in zip(y_pred,y)),Value(0.0))
    for p in parms:
        p.grad=0.0
    loss.backward()
    for p in parms:
        p.data+=(-0.01)*p.grad
